# CoinMetrics Case Study

Objective - to evaluate skills and abilities in multiple ways:
1. importing data
2. wrangling data
3. exploring data
4. analysis
5. modelling
6. communicating results

Provide:
1. A written explanation of how to approach the problem
2. Present the beginning phases of implementation using coin metrics data

Of the four options made available in the case study, option 3 was chosen

### Advocating for CoinMetric's data

Produce quality research that is of value to potential clients (doesn’t have to be complete) with a particular focus on network data

### Initial ideas

My first rough ideas were: 
1. comparing different Bitcoin based chains, (BTC, BCH, LTC, BSV) to test the influence of whales and compare this to their respective (evolving) claims to be a store of value (SoV) and/or alternative to cash.
2. developing and expanding some of the research by [Willy Woo](http://charts.woobull.com/bitcoin-hodl-waves/). I find his research to be outstanding. In particular: 
    1. days destroyed
    2. hodl waves
    3. thermo cap 
    4. average cap 
 
I think the following ideas are also interesting and worth investigating, but not possible within the scope of this exercise:
 
3. Tracking the number of twitter followers of various crypto-twitter thought leaders and celebrities to test the hypothesis that *"an increase in follower numbers shows that new retail investors are entering crypto-markets, and an increase in price is expected soon"*
 
 Thought leaders / crypto celebrities could be further grouped by what types of coins they speak about most  - smart contracts, DeFi, privacy coins, etc. 
 
 Weibo could be analysed as well as Twitter to understand Chinese markets, Korean twitter could be analysed for the Korean retail market, etc. 

4. I have an existing side project which has the goal of using a recurrent neural net to predict BTC price movements. The app (model, stored data, data pipeline, visualization of results) will run autonomously on Google Cloud Platform. Candle data is consumed from CoinAPI.io and stored in BigQuery. 

 Technical indicators will be calculated and used as additional factors to the model. Sentiment analysis from news outlets (Bloomberg, FT) would be added later. 

 The model would be written using TensorFlow, and the BigQuery tables names would use BQ's date format capabilites. This would make the project faster and cheaper. 

Idea 1 seemed like a sensible option. 

####  Testing the influence of whales and "normal users" on BTC and 4 BTC forks, and discussing results in the context of each chain's claimed technical advantages and use cases as e.g. a store of value or alternative to cash

This will be achieved by comparing daily mean USD transaction value to daily median USD transaction value. This is done by calculating the mean-median  ratio of transaction value (MMR).

__Hypothesis__: If a chain has a much smaller median transaction size than mean transaction size, then on chain activity is dominated not by regular users making normal daily transactions, but by whales moving large amounts of currency to artificially inflate usage metrics. 

This could contradict claims by that blockchain's community of being a popular form of digital cash. 

We assume that:
1. If a blockchain is functioning as digital cash, then most of its transactions would be small. 


2. Conversely, if a blockchain has relatively little organic use by normal users then whales (users with large holdings) will make up a large proportion of on-chain activity and would have average transaction sizes much larger than a day-to-day transaction.


3. Where the ratio of mean to median transaction value is relatively high, we have an environment where the mean value is much higher than the median value, which shows that daily total value transacted is dominated by a few relatively large transactions, rather than many small value transactions. This would show that whales dominate the blockchain (and likely market behavior) rather than members of the general public or retail investors. 

#### Chains:
The chains that will be analysed here are:
- BTC
- BCH 
- BSV 
- LTC 
- DOGE


#### Fields
using the coinmetrics api, the following metrics will be used:
1. _TxTfrValMeanUSD_ 

 The sum USD value of native units transferred divided by the count of transfers (i.e., the mean "size" in USD of a transfer) that interval.


2. *TxTfrValMedUSD*

 The median USD value transferred per transfer (i.e., the median "size" in USD of a transfer) that interval.


3. *TxTfrValUSD*

 The sum USD value of all native units transferred (i.e., the aggregate size in USD of all transfers) that interval.




In [28]:
# import and setup
import requests
import json

import cufflinks as cf
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio.plotly as py

init_notebook_mode(connected=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
def get_metricdata(asset_id, payload):
    url = f'https://community-api.coinmetrics.io/v2/assets/{asset_id}/metricdata'
    response = requests.get(
        url=url,
        params=payload
    )
    
    if response.status_code == 200:
        print(f'{asset_id} - success!')
        return json.loads(response.content.decode('utf-8'))
    else:
        print(f'status_code: {response.status_code}')
        return None

In [55]:
payload = {
    'metrics':  'PriceUSD,'+
                'TxTfrValMeanUSD,'+
                'TxTfrValMedUSD,'+
                'TxTfrValUSD',
    'start': '2016-01-01',
}

asset_list = ['btc', 'ltc', 'bch', 'bsv', 'doge']
data = {}
for asset in asset_list:
    data[asset] = get_metricdata(asset, payload)

btc - success!
ltc - success!
bch - success!
bsv - success!
doge - success!


In [56]:
dataframes = {}
cols = ['PriceUSD', 'TxTfrValMeanUSD', 'TxTfrValMedUSD', 'TxTfrValUSD']
for asset in data.keys():
    values = [ each['values'] for each in data[asset]['metricData']['series']]
    index = [ each['time'] for each in data[asset]['metricData']['series']]
    
    df = pd.DataFrame.from_records(values, columns = cols)
    df.index = pd.to_datetime(index, infer_datetime_format=True).date
    
    for col in df.columns:
        df[col] = df[col].astype(float)
    
    df['TxCount'] = df.TxTfrValUSD / df.TxTfrValMeanUSD
    df['MeanMedianRatio'] = df.TxTfrValMeanUSD / df.TxTfrValMedUSD
    
    dataframes[asset] = df

In [57]:
dataframes['bsv'].sample(5)
dataframes['btc'].sample(5)
dataframes['doge'].sample(5)

,PriceUSD,TxTfrValMeanUSD,TxTfrValMedUSD,TxTfrValUSD,TxCount,MeanMedianRatio
2019-02-15,61.706587,1516.754318,61.706587,8.353676e+07,55076.0,24.580104
2019-02-19,66.775855,9736.902716,421.828343,1.674845e+08,17201.0,23.082618
2019-03-08,64.837298,7438.208012,0.069422,8.554683e+07,11501.0,107144.911292
2019-03-28,63.633667,4749.332321,2.107293,6.684210e+07,14074.0,2253.759666
2019-03-06,66.241482,8302.813849,2.659609,4.090796e+07,4927.0,3121.817000


,PriceUSD,TxTfrValMeanUSD,TxTfrValMedUSD,TxTfrValUSD,TxCount,MeanMedianRatio
2018-01-12,13770.848818,19686.191156,344.271220,1.684626e+10,855740.0,57.182216
2016-04-15,430.724419,1885.295034,12.001705,9.001021e+08,477433.0,157.085598
2016-06-20,722.497452,2395.496580,12.204991,1.476701e+09,616449.0,196.271892
2017-10-08,4607.925936,9428.789627,53.004327,5.515022e+09,584913.0,177.887168
2017-01-03,1032.609053,2199.476710,15.342221,1.653857e+09,751932.0,143.361034


,PriceUSD,TxTfrValMeanUSD,TxTfrValMedUSD,TxTfrValUSD,TxCount,MeanMedianRatio
2016-03-05,0.000217,164.291196,0.008380,9.729160e+06,59219.0,19604.589277
2019-07-16,0.002774,460.318149,0.264234,3.410957e+07,74100.0,1742.081946
2016-02-04,0.000285,177.023858,0.004270,1.199992e+07,67787.0,41455.115445
2017-05-23,0.003468,7038.797105,7.092899,2.823191e+08,40109.0,992.372452
2019-03-23,0.002011,595.654654,0.497619,4.381814e+07,73563.0,1197.008547


### Compare absolute mean and median daily USD transaction value for BTC since January 2016

In [60]:
btc_mean = go.Scatter(
    x=dataframes['btc'].index,
    y=dataframes['btc'].TxTfrValMeanUSD,
    name='BTC mean',
    #marker=dict(color='#ffcdd2')
)
btc_median = go.Scatter(
    x=dataframes['btc'].index,
    y=dataframes['btc'].TxTfrValMedUSD,
    name='BTC median'
    #marker=dict(color='#ffcdd2')
)

data = [btc_mean, btc_median]
layout = go.Layout(
    title="BTC median and mean transaction values by day",
    xaxis=dict(title='Date'),
    yaxis=dict(title='USD value'),
    yaxis_type='log'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig) 

### Plot the ratio of daily mean to median USD transaction values for each asset since January 2016

In [61]:
def scatter_plot(asset, name):
    return go.Scatter(
        x=dataframes[asset].index,
        y=dataframes[asset].MeanMedianRatio,
        name=name
    )
        
data = [scatter_plot(asset, asset) for asset in dataframes.keys()]
layout = go.Layout(
    title="Ratio of daily mean to median transaction value",
    xaxis=dict(title='Date'),
    yaxis=dict(title='Ratio'),
    yaxis_type='log'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig) 

### Conclusions

The chart above shows that BTC has the lowest ratio of mean to median daily transaction value. This suggests that it has relatively strong organic use compared to the other blockchains, and is less influenced by whales than the other blockchains on the chart. Its MMR has less day-to-day variation that the other chains, suggesting a wider and more established user base. This should be further analysed by considering total daily transaction volumes. 


Using this ratio as a proxy to measure organic use, the chain with the second most organic use is Litecoin.


Since the start of 2019, Dogecoin influence of whales has been decreasing

Of the two contentious hard forks, Bitcoin cash shows two distinct phases with different characterists in each. From its inception in August 2017 to November 2018, the influence of whales and inorganic behaviour increased at a steady rate. Notably at the coins genesis, there appears to have been a large organic user base transacting daily, bringing the median transaction value to within 50 - 100x the mean daily transaction value. This was lower than Bitcoins, which had a much more consistent but higher ratio of 120 - 200.


After November 10 2018, the ratio increases from an average of approximately 500 to approximately 10,000. This is a stark and abrupt change in the daily ratio, and suggests that either organic use drastically decreased or it (very suddenly) started being used for very large value transfers by relatively few users.


Dogecoin, despite its status as a "joke" blockchain, appears to have more widespread organic use than either BCH or BSV, but less than BTC or LTC. Additionally, even though Elon Musk has professed to be a [fan](https://twitter.com/elonmusk/status/1113009339743100929) , it seems unlikely that he is using it to transmit a large fraction of his personal wealth.

### Next Steps
1. This brief investigation was developed over the course of an afternoon, in line with the project brief. In order to be applied in a commercial context it should be expanded and tested in at least the following ways:
 1. Test if the central assumption of this analysis is true. Possible approaches could include: 
        - Exchange outflows should be filtered out. Could this be done using known exchange addresses (exchanges aggregate organic retail investor behavior)?
        - What is the influence of "change" transactions - in aggregate this should be nil.
        - For BTC and LTC, are the lightning networks distorting the results significantly by hiding organic low value activity?
        - For BTC, is the [liquid sidechain](https://blockstream.com/liquid-faq/) hiding the activity of whales to the extent that it is not the "healthiest" of the 5 blockchains analysed?

 1. Can we infer where the whales and general users live by analysing the time of transactions? People are much more likely to make a transaction at midday than midnight, and we could use this to investigate geographic clusterings. Is BTC a "western" chain, whilst BCH has more organic use in Asia? 
 
 1. An analysis of daily transaction volume (in USD terms) would provide useful background and context in which to interpret the significance of differences between each chain.
 
 1. Similarly, comparing the hash power dedicated to mining new blocks on each chain would indicate commercial interests, and abrupt changes in hash power could possibly be correlated with changes in mean-median ratio (MMR).